In [ ]:
# extract top ranked book id's, etc from url

In [150]:
from sudachipy import tokenizer
from sudachipy import dictionary
import sqlite3
import jaconv
import re
import requests
from requests.exceptions import HTTPError
import json
import time
import traceback
import os

# sqlite example

In [5]:
import sqlite3


con = sqlite3.connect("jmdict.db")

cur = con.cursor()

for row in cur.execute('SELECT furigana FROM jmdict where word = "儀式";'):
    print(row)

con.close()

('0:ぎ;1:しき',)


# jaconv example

In [1]:
import jaconv

# Hiragana to Katakana
jaconv.hira2kata('ともえまみ')
# => 'トモエマミ'

# Hiragana to half-width Katakana
jaconv.hira2hkata('ともえまみ')
# => 'ﾄﾓｴﾏﾐ'

# Katakana to Hiragana
jaconv.kata2hira('巴マミ')

'巴まみ'

# DB to hold Aozora Stats

In [104]:
wordDict = {"kango": ["a", "銀河鉄道"], "wago": ["走れメロス", "b"], "verb": ["走れメロス", "b"],}
wordListJson = json.dumps(wordDict, indent = None, ensure_ascii=False)
print(wordListJson)

{"kango": ["a", "銀河鉄道"], "wago": ["走れメロス", "b"], "verb": ["走れメロス", "b"]}


In [105]:
bookId = 1
title = "test title"
authors = "me, you"
exData = [bookId, title, authors, wordListJson, 4.55]

In [115]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
cur2.execute("DROP TABLE IF EXISTS aozoraStats")

cur2.execute("CREATE TABLE aozoraStats(bookid INTEGER PRIMARY KEY, \
            title TEXT, authors TEXT, wordlist JSON, difficulty FLOAT)")


In [106]:
cur2.execute("INSERT INTO aozoraStats (bookid, title, authors, wordlist, difficulty) VALUES (?, ?, ?, ?, ?);", exData)
con2.commit()

In [117]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
for row in cur2.execute('SELECT * FROM aozoraStats;'):
    print(row)
    jsonResponse = json.loads(row[3])
    print(jsonResponse["verb"])
con2.close()

(127, '羅生門', '芥川 竜之介', '{"kango": ["日", "下人", "門", "所々", "円柱", "一", "大路", "以上", "市", "烏帽子", "二三", "人", "もの", "何故", "地震", "火事", "饑饉", "災", "洛中", "旧記", "仏像", "仏具", "丹", "金銀", "箔", "木", "料", "始末", "修理", "狐狸", "死人", "習慣", "気味", "近所", "足ぶみ", "何", "羽", "輪", "鴟尾", "胡麻", "勿論", "上", "肉", "刻限", "目", "糞", "点々", "一番", "紺", "頬", "気", "作者", "格別", "主人", "家", "前", "当時", "町", "衰微", "永年", "実", "余波", "途方", "朝", "影響", "明日", "次第", "屋根", "斜", "手段", "築", "土", "死", "何度", "道", "低徊", "逢着", "肯定", "当然", "勇気", "大儀", "遠慮", "晩", "楽", "楼", "眼", "下", "分", "中段", "身", "容子", "火", "蜘蛛", "巣", "天井", "急", "平", "内", "噂", "死骸", "無造作", "範囲", "数", "人間", "事実", "人形", "口", "部分", "一層", "永久", "腐爛", "臭気", "感情", "嗅覚", "背", "老婆", "松", "木片", "顔", "多分", "六", "恐怖", "四", "好奇心", "暫時", "呼吸", "記者", "語", "頭身", "毛", "両手", "丁度", "子", "心", "同時", "憎悪", "語弊", "悪", "反感", "問題", "未練", "善悪", "自分", "一目", "無言", "勝敗", "無理", "眼球", "執拗", "明白", "生死", "全然", "意志", "支配", "意識", "仕事", "円満", "成就", "得意", "満足", "声", "検非違使", "庁", "役人", "旅", "肉食", "物", "存外", "平凡", "失望"

In [3]:
# find top ranked books by year/month

def fetchRankedBooks(date = "2020/01"):
    try:
        response = requests.get(f'https://pubserver2.herokuapp.com/api/v0.1/ranking/txt/{date}')
        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        return jsonResponse

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [8]:
def fetchURL(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        # note this query doesn't contain json
        if response is not None and response.text is not None:
            return response.text
        else:
            return None

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [14]:
%time fetchRankedBooks()

Entire JSON response
Wall time: 4.8 s


In [13]:
rankedBooks = fetchRankedBooks()
for row in rankedBooks:
    authors = ", ".join(row["authors"])
    print("book id " + str(row["book_id"]) + " title " + row["title"] + " authors " + authors)


book id 773 title こころ authors 夏目 漱石
book id 301 title 人間失格 authors 太宰 治
book id 127 title 羅生門 authors 芥川 竜之介
book id 456 title 銀河鉄道の夜 authors 宮沢 賢治
book id 789 title 吾輩は猫である authors 夏目 漱石
book id 2093 title ドグラ・マグラ authors 夢野 久作
book id 57624 title さぶ authors 山本 周五郎
book id 752 title 坊っちゃん authors 夏目 漱石
book id 1465 title 蟹工船 authors 小林 多喜二
book id 92 title 蜘蛛の糸 authors 芥川 竜之介
book id 1565 title 斜陽 authors 太宰 治
book id 57228 title 怪人二十面相 authors 江戸川 乱歩
book id 57817 title 日本婦道記 authors 山本 周五郎
book id 1567 title 走れメロス authors 太宰 治
book id 424 title 檸檬 authors 梶井 基次郎
book id 794 title 三四郎 authors 夏目 漱石
book id 799 title 夢十夜 authors 夏目 漱石
book id 45630 title 〔雨ニモマケズ〕 authors 宮沢 賢治
book id 2078 title 舞姫 authors 森 鴎外
book id 49866 title 変身 authors カフカ フランツ
book id 56648 title 人間椅子 authors 江戸川 乱歩
book id 43737 title 銀河鉄道の夜 authors 宮沢 賢治
book id 776 title 草枕 authors 夏目 漱石
book id 5016 title 源氏物語 authors 紫式部 
book id 52409 title 三国志 authors 吉川 英治
book id 56650 title Ｄ坂の殺人事件 authors 江戸川 乱歩
book

# Sudachi Python Parser Example

In [15]:
# Create the Sudachi Parser.
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C # mode.C is the one that splits things into the smallest form

In [125]:
# this cell is just used to test the parser and help with debugging.

text = "※二十年戦考え二ヶ月べんきょう花火花粉たくさんきれいな寿司が食べたい。勉強をします。音楽がすきですから、よく CD を聞きます。ワンさん"

tokens = tokenizer_obj.tokenize(text, mode)
for token in tokens:
    print("surface {}, dict {}, reading {}, pos {}"\
          .format(token.surface(), token.dictionary_form(), token.reading_form(), token.part_of_speech()))

surface ※, dict ※, reading キゴウ, pos ('補助記号', '一般', '*', '*', '*', '*')
surface 二十, dict 二十, reading ニジュウ, pos ('名詞', '数詞', '*', '*', '*', '*')
surface 年, dict 年, reading ネン, pos ('名詞', '普通名詞', '助数詞可能', '*', '*', '*')
surface 戦, dict 戦, reading セン, pos ('接尾辞', '名詞的', '一般', '*', '*', '*')
surface 考え, dict 考える, reading カンガエ, pos ('動詞', '一般', '*', '*', '下一段-ア行', '連用形-一般')
surface 二, dict 二, reading ニ, pos ('名詞', '数詞', '*', '*', '*', '*')
surface ヶ月, dict ヶ月, reading カゲツ, pos ('名詞', '普通名詞', '助数詞可能', '*', '*', '*')
surface べんきょう, dict べんきょう, reading ベンキョウ, pos ('名詞', '普通名詞', 'サ変可能', '*', '*', '*')
surface 花火, dict 花火, reading ハナビ, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface 花粉, dict 花粉, reading カフン, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface たくさん, dict たくさん, reading タクサン, pos ('副詞', '*', '*', '*', '*', '*')
surface きれい, dict きれい, reading キレイ, pos ('形状詞', '一般', '*', '*', '*', '*')
surface な, dict だ, reading ナ, pos ('助動詞', '*', '*', '*', '助動詞-ダ', '連体形-一般')
surface 寿司, dict 寿司, reading

In [96]:
# Removes ruby and some other inserted stuff from the Aozora text that isn't part of the "actual" text itself.
def CleanUPAozoraText(text):
    rubyRegex = """(《.*?》)"""
    text = re.sub(rubyRegex, '', text)
    dashes = "-------------------------------------------------------"
    dashExpression = "(" + dashes + ".*" + dashes + ")"
    text = re.sub(dashExpression, '', text, 0, re.DOTALL)
    indentRegex = """(［＃.*?］)"""
    text = re.sub(indentRegex, '', text,)
    creditsRegex = "(底本：.*)"
    text = re.sub(creditsRegex, '', text, 0, re.DOTALL)
    return text

# Magic to determine Wago vs Kango

In [11]:
def CalcTrueLen(furigana):
    # Function to return the true length of the reading.
    # Certain reading combinations can have multi characters but only count as 1 sound.
    # This likely indicates Kango. This function accounts for those combinations.
    # This link has good info for understanding the logic below. 
    # https://japanese.stackexchange.com/questions/41585/heuristics-for-identifying-japanese-origin-words-%E5%92%8C%E8%AA%9E
    vowels = ["あ", "い", "う", "え", "お",]
    glides1 = ["ょ", "ゅ", "ゃ"]
    glides2 = ["ょう", "ゅう"]
    furiLen = len(furigana)
    if furiLen == 1:
        return 1
    elif furiLen == 2:
        lastChar = furigana[1]
        if "しき" == furigana:
            return 1
        elif lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち":
            return 1
        elif lastChar in vowels:
            return 1
        elif lastChar in glides1:
            return 1
        else:
            return 2
    elif furiLen == 3:
        lastTwo = furigana[1:]
        secondChar = furigana[1]
        lastChar = furigana[2]
        if lastTwo in glides2:
            return 1
        elif not (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            # Second char in the last but last char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        elif (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and not (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            # Last char in the list but second char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        else:
            return 3
    elif furiLen == 4:
        secondChar = furigana[1]
        lastChar = furigana[3]
        if (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            return 2
        else:
            return furiLen
    else:
        #just in case
        return furiLen
    

# Method to extract stats from text

In [168]:
# All of the math below is meant to match with the math used by JReadability
def ExtractStats(tokens, text, includeAuxVerbs = False):
    numVerbs = 0
    numAux = 0
    numAuxVerbs = 0
    numKango = 0
    numWago = 0
    numGairaigo = 0
    sentenceAvgLen = 0
    numPeriods = 0
    numSpaces = 0
    verb = "動詞"
    dependent = "非自立可能"
    aux1 = "助動詞" # counts as an auxVerb
    aux2 = "助詞" # counts as an auxVerb
    auxVerb = "格助詞" # counts as an auxVerb
    adverb = "副詞"
    conjuction = "接続詞"
    symbol = "補助記号"
    space = "空白"
    naAdj = "形状詞"
    iAdj = "形容詞"
    noun = "名詞"
    nounLike = "名詞的"
    properNoun = "固有名詞"
    pronoun = "代名詞"
    counter = "助数詞可能"
    connectingWord = "連体詞"
    suruVerb = "サ変可能"
    suffix = "接尾辞"
    hiraganaUniLow = u'\u3040'
    katakanaUniLow = u'\u30A0' 
    katakanaUniHi = u'\u30FF'
    kangoExceptionList = ["きれい"]
    alphabetRegexp = re.compile(r'[a-zA-Z]')
    numberRegexp = re.compile(r'[0-9]')
    kangoWords = {}
    wagoWords = {}
    verbWords = {}
    gairaigoWords = {}
    totalVerbsList = []
    
    
    for token in tokens:
        #print(token.surface())
        if token.surface() == "。":
            numPeriods += 1
            continue
        
        
        posList = token.part_of_speech()
        if aux2 in posList or auxVerb in posList:
            numAux +=1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif aux1 in posList:
            numAuxVerbs += 1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
            if includeAuxVerbs:
                # Not sure if auxVerbs should count in the number of total auxillary words.
                # If includeAuxVerbs then add it to the total
                numAux +=1
        elif verb in posList:
            dictForm = token.dictionary_form()
            #JReadability doesn't consider する/いる in the calculation.
            if dependent not in posList:
                numVerbs += 1
                verbWords[dictForm] = 1
                totalVerbsList.append(dictForm)
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif naAdj in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            if reading in kangoExceptionList:
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
            elif reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif adverb in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
            if reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif conjuction in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif iAdj in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif properNoun in posList:
            # JReadability doesn't count these as wago or kango.
            continue
        elif suffix in posList and not nounLike in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif pronoun in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif counter in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif connectingWord in posList:
            continue
            #numWago += 1
            #wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif noun in posList or nounLike in posList:
            #complicated case
            surface = token.surface()
            #print("found noun. posList {}".format(posList))
            if alphabetRegexp.search(surface):
                continue
            if numberRegexp.search(surface):
                continue
            elif suruVerb in posList:
                # this checks for nouns that can take the suffix する to become verbs. Those nouns are usually Kango.
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(surface))
            elif katakanaUniLow <= min(surface) <= katakanaUniHi:
                # considering katakana nouns as gairaigo
                numGairaigo += 1
                gairaigoWords[surface] = 1
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi and not (hiraganaUniLow <= max(surface) <= katakanaUniHi):
                #word contains hiragana/katakana + kanji. Most likely wago
                numWago += 1
                wagoWords[token.surface()] = 1
            else:
                # Need to look at the reading compared to the surface to determine the character length
                # This is a hueristic so it's certainly not too accurate
                reading = jaconv.kata2hira(token.reading_form())
                #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
                if reading == surface:
                    # probably a hiragana only word, so wago
                    numWago += 1
                    wagoWords[surface] = 1
                    #print("adding {} to numWago".format(surface))
                    continue
                
                query = 'SELECT furigana FROM jmdict where word = "{}" and reading = "{}";'\
                                  .format(surface, reading)
                #print(query)
                row = cur.execute(query).fetchone()
                if row != None:
                    # Need deal with special cases ん、つ、く、しき
                    try:
                        #print(f"found query result in db {row[0]}")
                        readingLen = 0
                        furiHelper = row[0].split(";") #e.g. '0:ぎ;1:しき'
                        for item in furiHelper:
                            ruby = item.split(":")
                            if "-" not in ruby[0]:
                                #normal case
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                            else:
                                #special case like with 三和土|たたき|0-2:たたき
                                numRange = ruby[0].split("-")
                                # Is there a need to do something special with the numRange??
                                # Doing nothing for now.
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                        
                        #again this is a hueristic and won't always be true
                        #print(f'surface len {len(surface)}, reading len {readingLen}')
                        if len(surface) == readingLen:
                            numKango += 1
                            kangoWords[surface] = 1
                            #print("adding {} to numKango".format(surface))
                        else:
                            numWago += 1
                            wagoWords[surface] = 1
                            #print("adding {} to numWago".format(surface))
                                  
                    except Exception as err:
                        print(f'Error occurred: {err}')
                else:
                    #print(f"couldn't find {surface} {reading}")
                    # word not found in jmdictfurigana so just use the reading
                    if len(surface) == len(reading):
                        numKango += 1
                        kangoWords[surface] = 1
                        #print("adding {} to numKango".format(surface))
                    else:
                        numWago += 1
                        wagoWords[surface] = 1
                        #print("adding {} to numWago".format(surface))
             
        elif space in posList:
            numSpaces += 1
        else:
            # It's probably a symbol. Do nothing
            #print("What dis? {}".format(posList))
            continue
            
    # subtracting the numPeriods from the number of tokens before the divide would make this more accurate
    # but that's not what JReadability does, so leaving it this way for now.
    numTokens = len(tokens) - numSpaces
    # some text in Aozora don't use periods such as poems. Some using single newlines and some use double newlines
    if numPeriods == 0:
        # Check for newlines like in a poem.
        numSingle = text.count(os.linesep)
        numDouble = text.count(os.linesep + os.linesep)
        print(f"numSingle {numSingle}, numDouble {numDouble}, use double {round(numSingle/max(1, numDouble)) == 2}")
        if round(numSingle/max(1, numDouble)) == 2:
            # text likely uses double newlines since the number of single newlines is roughly 2x
            print("using num of double newlines since no periods found")
            numPeriods = numDouble
        else:
            # hard to say, but likely single newlines.
            print("using num of newlines since no periods found")
            numPeriods = numSingle
    
    sentenceAvgLen = numTokens/ max(1, numPeriods)
    uniqueKango = list(kangoWords.keys())
    uniqueWago = list(wagoWords.keys())
    uniqueVerbs = list(verbWords.keys())
    print(f"unique words: kango {len(uniqueKango)}, wago {len(uniqueWago)}, verbs {len(uniqueVerbs)}")
    
    #print(f"{len(kangoWords.keys())} unique kango words found {kangoWords.keys()}")
    #print(f"{len(wagoWords.keys())} unique wago words found {wagoWords.keys()}")
    #print(f"{len(gairaigoWords.keys())} unique gairaigo words found {gairaigoWords.keys()}")
    #print(f"{len(verbWords.keys())} unique verb words found {verbWords.keys()}")
    #print(f"{len(totalVerbsList)} total verb words found {totalVerbsList}")
    print("result avgLen {}, kango {}, wago {}, gairaigo {} verbs {}, auxiliary {}, auxVerbs {}"\
          .format(round(sentenceAvgLen, 2), numKango, numWago, numGairaigo, numVerbs, numAux, numAuxVerbs))
    return (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, \
            uniqueKango, uniqueWago, uniqueVerbs)

# Calc Difficulty

In [13]:
def CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens):
    # Linear Regression Formula from http://jhlee.sakura.ne.jp/papers/lee-et-al2016rb.pdf
    # X = { 平 均 文 長 *-0.056}+{ 漢 語 率 *-0.126}+{ 和 語 率*-0.042}+{動詞率*-0.145}+{助詞率*-0.044}+11.724
    percentKango = numKango / numTokens * 100
    percentWago = numWago / numTokens * 100
    percentVerbs = numVerbs / numTokens * 100
    percentAuxVerbs = numAux / numTokens * 100
    difficulty = sentenceAvgLen*-0.056 + percentKango*-0.126 + percentWago*-0.042 \
        + percentVerbs*-0.145 + percentAuxVerbs*-0.044 + 11.724
    
    return difficulty

# Cell to test the extraction of text stats and compute difficulty (for debug)

In [90]:
# Same logic used in the real extraction for Aozora text, but pulled out into a more easily testable format for debugging.

# connect to jmdictfurigana
con = sqlite3.connect("dbOutput/jmdict.db")
cur = con.cursor()
# call to Aozora goes here...

#text = "音楽がすきですから、よく CD を聞きます。日本がすきですから、日本語を勉強します。安かったですから、買いました。ディズニーランドは楽しかったです。教室は静かでした。わたしはラーメンがすきです。わたしはたばこがきらいです。ワンさんは日本語が上手です。わたしは料理が下手です。"
text = "寿司が食べたい。"
#text = "わたしは料理が下手です。"
#text = "スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。"

tokens = tokenizer_obj.tokenize(text, mode)
(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) = ExtractStats(tokens, text)

difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
print("difficulty = {}".format(difficulty))

#Store the difficulty and other stats in sqlite

#finally close the connection to jmdictfurigana
con.close()

unique words: kango 1, wago 3, verbs 1
result avgLen 5.0, kango 1, wago 3, gairaigo 0 verbs 1, auxiliary 1, auxVerbs 1
difficulty = 2.6239999999999988


In [189]:
# just for testing.
katakanaUniLow = u'\u30A0' 
katakanaUniHi = u'\u30FF'
word = "ヶ"
if katakanaUniLow <= min(word) <= katakanaUniHi:
    print("katakana")
else:
    print("not katakana")

katakana


In [153]:
# retry with modifications?? # delete this later
CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)

1.6179066974595848

In [54]:
# unicode for kanji = [\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff]

# Grade Aozora Content

In [ ]:
longBooks = []

In [127]:
start = time.time()
try:
    # connect to dbs
    con = sqlite3.connect("dbOutput/jmdict.db")
    cur = con.cursor()
    con2 = sqlite3.connect('dbOutput/aozoraStats.db')
    cur2 = con2.cursor()
    rankedBooks = fetchRankedBooks()
    for row in rankedBooks:
        startTimeThisBook = time.time()
        authors = ", ".join(row["authors"])
        print("\nbook id " + str(row["book_id"]) + " title " + row["title"] + " authors " + authors)

        
        bookId = row["book_id"]
        #check if the book was already scored.
        result = cur2.execute(f'SELECT bookid FROM aozoraStats where bookid = {bookId};').fetchone()
        if result is not None:
            print(f"{bookId} was already added to the db. Skipping it.")
            continue
        
        # call to Aozora for book text
        url = f"https://pubserver2.herokuapp.com/api/v0.1/books/{bookId}/content?format=txt"
        text = fetchURL(url)
        if text is None:
            print(f"failed to get data for book id {bookId}")
        else:
            text = CleanUPAozoraText(text)
            bookLen = len(text.encode('utf-8'))
            sudachiTokenLimit = 49149
            if bookLen > sudachiTokenLimit:
                print(f"Book {bookId} is length {bookLen} which exceeds the SudachiPy Tokenizer limit {sudachiTokenLimit}." +\
                      "Skipping it.")
                longBooks.append(bookId)
                continue
            else:
                tokens = tokenizer_obj.tokenize(text, mode)
                (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) \
                    = ExtractStats(tokens, text)

                difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
                print("text len {}, difficulty = {}".format(bookLen, round(difficulty, 2)))

                # Create json unique words obj
                uniqueWordDict = {"kango": uniqueKango, "wago": uniqueWago, "verb": uniqueVerbs}
                wordListJson = json.dumps(uniqueWordDict, indent = None, ensure_ascii=False)
                bookStats = [row["book_id"], row["title"], authors, wordListJson, round(difficulty, 2)]
                
                #Store the difficulty and other stats in sqlite
                cur2.execute("INSERT INTO aozoraStats (bookid, title, authors, wordlist, difficulty) VALUES (?, ?, ?, ?, ?);",\
                             bookStats)
                con2.commit()
                endTimeThisBook = time.time()
                print(f"{endTimeThisBook - startTimeThisBook} seconds to process book {bookId}.")
                print(f"Elapsed time is {round((endTimeThisBook - start)/60, 2)} minutes")
                
                
    print(f"{len(longBooks)} long books were skipped.")

    
except Exception as err:
    print(f'Error occurred: {err}')
    print(traceback.format_exc())
finally:
    #finally close the connection to jmdictfurigana
    con.close()
    con2.close()
    end = time.time()
    print(f"total time {round((end - start)/60, 2)} minutes")



book id 773 title こころ authors 夏目 漱石
Book 773 is length 486365 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 301 title 人間失格 authors 太宰 治
Book 301 is length 220846 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 127 title 羅生門 authors 芥川 竜之介
127 was already added to the db. Skipping it.

book id 456 title 銀河鉄道の夜 authors 宮沢 賢治
Book 456 is length 116360 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 789 title 吾輩は猫である authors 夏目 漱石
Book 789 is length 961808 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 2093 title ドグラ・マグラ authors 夢野 久作
Book 2093 is length 1282250 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 57624 title さぶ authors 山本 周五郎
Book 57624 is length 53743 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 752 title 坊っちゃん authors 夏目 漱石
Book 752 is length 266813 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 1465 t

Book 56645 is length 52535 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 56646 title 心理試験 authors 江戸川 乱歩
Book 56646 is length 68415 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 1927 title 注文の多い料理店 authors 宮沢 賢治
unique words: kango 104, wago 568, verbs 106
result avgLen 16.9, kango 209, wago 2559, gairaigo 23 verbs 225, auxiliary 962, auxVerbs 389
text len 16745, difficulty = 4.87
1.170396089553833 seconds to process book 1927.
Elapsed time is 1.15 minutes

book id 258 title グッド・バイ authors 太宰 治
unique words: kango 470, wago 1161, verbs 308
result avgLen 17.25, kango 892, wago 5918, gairaigo 130 verbs 571, auxiliary 2444, auxVerbs 755
text len 45190, difficulty = 4.86
1.5320937633514404 seconds to process book 258.
Elapsed time is 1.17 minutes

book id 42618 title 桜の森の満開の下 authors 坂口 安吾
Book 42618 is length 50708 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 462 title 風の又三郎 authors 宮沢 賢治
unique words: kango 27

Book 52414 is length 488441 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 270 title 富嶽百景 authors 太宰 治
unique words: kango 499, wago 1205, verbs 288
result avgLen 22.95, kango 909, wago 6901, gairaigo 49 verbs 643, auxiliary 2747, auxVerbs 877
text len 45370, difficulty = 4.29
1.5920372009277344 seconds to process book 270.
Elapsed time is 2.14 minutes

book id 605 title モルグ街の殺人事件 authors ポー エドガー・アラン
Book 605 is length 106321 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 179 title 藪の中 authors 芥川 竜之介
unique words: kango 170, wago 745, verbs 216
result avgLen 23.04, kango 350, wago 4266, gairaigo 0 verbs 409, auxiliary 1624, auxVerbs 697
text len 25628, difficulty = 4.11
1.30000638961792 seconds to process book 179.
Elapsed time is 2.18 minutes

book id 59056 title お前は此の頃よくねむる authors 中野 鈴子
unique words: kango 20, wago 163, verbs 38
result avgLen 465.0, kango 24, wago 431, gairaigo 0 verbs 49, auxiliary 142, auxVerbs 64
text len 2537, di

unique words: kango 58, wago 454, verbs 103
result avgLen 24.43, kango 160, wago 2713, gairaigo 4 verbs 240, auxiliary 1003, auxVerbs 453
text len 16603, difficulty = 4.46
1.2585854530334473 seconds to process book 52973.
Elapsed time is 3.22 minutes

book id 52421 title 私本太平記 authors 吉川 英治
Book 52421 is length 376264 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 57181 title 赤い部屋 authors 江戸川 乱歩
Book 57181 is length 54615 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 59057 title ふしあわせ者のうた authors 中野 鈴子
unique words: kango 19, wago 80, verbs 16
result avgLen 182.0, kango 26, wago 149, gairaigo 1 verbs 19, auxiliary 59, auxVerbs 13
text len 1053, difficulty = -6.65
1.0468206405639648 seconds to process book 59057.
Elapsed time is 3.29 minutes

book id 682 title 舞姫 authors 森 鴎外
unique words: kango 609, wago 1647, verbs 470
result avgLen 30.88, kango 1250, wago 7929, gairaigo 29 verbs 923, auxiliary 3344, auxVerbs 884
text len 46666, diffi

Book 772 is length 70559 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 43016 title トロッコ authors 芥川 竜之介
unique words: kango 133, wago 493, verbs 130
result avgLen 23.4, kango 269, wago 2076, gairaigo 41 verbs 242, auxiliary 851, auxVerbs 252
text len 13219, difficulty = 3.79
1.1248862743377686 seconds to process book 43016.
Elapsed time is 4.11 minutes

book id 58875 title みにくいアヒルの子 authors アンデルセン ハンス・クリスチャン
unique words: kango 82, wago 917, verbs 213
result avgLen 24.18, kango 289, wago 5742, gairaigo 205 verbs 468, auxiliary 2192, auxVerbs 970
text len 40381, difficulty = 4.87
1.0956740379333496 seconds to process book 58875.
Elapsed time is 4.13 minutes

book id 49009 title 草津から伊香保まで authors 田山 花袋, 田山 録弥
unique words: kango 74, wago 258, verbs 60
result avgLen 23.21, kango 92, wago 764, gairaigo 2 verbs 79, auxiliary 323, auxVerbs 88
text len 4218, difficulty = 3.48
0.951932430267334 seconds to process book 49009.
Elapsed time is 4.14 minutes

book id 919 ti

...

unique words: kango 31, wago 410, verbs 89
result avgLen 20.46, kango 59, wago 1634, gairaigo 2 verbs 144, auxiliary 591, auxVerbs 221
text len 11361, difficulty = 5.07
1.0901463031768799 seconds to process book 42311.
Elapsed time is 7.88 minutes

book id 49863 title 審判 authors カフカ フランツ
Book 49863 is length 679392 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 49966 title チベット旅行記 authors 河口 慧海
Book 49966 is length 93047 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 56750 title 益子の絵土瓶 authors 柳 宗悦
Book 56750 is length 116893 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 57543 title いさましい話 authors 山本 周五郎
Book 57543 is length 111921 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 545 title 夫婦善哉 authors 織田 作之助
Book 545 is length 92271 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 45761 title 河童 authors 芥川 竜之介
Book 45761 is length 109906 which exceeds the SudachiPy

unique words: kango 169, wago 426, verbs 111
result avgLen 37.21, kango 260, wago 1612, gairaigo 4 verbs 140, auxiliary 659, auxVerbs 205
text len 9559, difficulty = 2.7
1.1434364318847656 seconds to process book 98.
Elapsed time is 9.01 minutes

book id 2231 title キチガイ地獄 authors 夢野 久作
Book 2231 is length 70829 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 56076 title 平の将門 authors 吉川 英治
Book 56076 is length 705596 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 56754 title 新書太閤記 authors 吉川 英治
Book 56754 is length 565807 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 57786 title 樅ノ木は残った authors 山本 周五郎
Book 57786 is length 440288 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 52359 title つゆのあとさき authors 永井 荷風
Book 52359 is length 225842 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 621 title 名人伝 authors 中島 敦
unique words: kango 315, wago 756, verbs 244
result avgL

HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/56759/content?format=txt
failed to get data for book id 56759

book id 464 title 猫の事務所 authors 宮沢 賢治
unique words: kango 162, wago 578, verbs 127
result avgLen 18.49, kango 379, wago 2772, gairaigo 37 verbs 240, auxiliary 1049, auxVerbs 519
text len 19102, difficulty = 4.52
1.13673734664917 seconds to process book 464.
Elapsed time is 10.07 minutes

book id 45381 title 母を尋ねて三千里 authors アミーチス エドモンド・デ
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/45381/content?format=txt
failed to get data for book id 45381

book id 56868 title 盲目物語 authors 谷崎 潤一郎
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/56868/content?format=txt
failed to get data for book id 56868

book id 57670 title ちいさこべ authors 山本 周五郎
HTTP error occurred: 404 Client Error: Not Found for url: https://pubser

HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/43056/content?format=txt
failed to get data for book id 43056

book id 57505 title 踊る一寸法師 authors 江戸川 乱歩
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/57505/content?format=txt
failed to get data for book id 57505

book id 58447 title 大唐田または唐干田という地名 authors 柳田 国男
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/58447/content?format=txt
failed to get data for book id 58447

book id 60081 title 西洋科学は素晴らしい authors スミス コードウェイナー
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/60081/content?format=txt
failed to get data for book id 60081

book id 3827 title 或社会主義者 authors 芥川 竜之介
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/3827/content?format=txt
failed to 

HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/52430/content?format=txt
failed to get data for book id 52430

book id 57856 title 二十四の瞳 authors 壺井 栄
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/57856/content?format=txt
failed to get data for book id 57856

book id 58019 title 新編忠臣蔵 authors 吉川 英治
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/58019/content?format=txt
failed to get data for book id 58019

book id 767 title 人生 authors 夏目 漱石
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/767/content?format=txt
failed to get data for book id 767

book id 4261 title マリー・ロジェエの怪事件 authors ポー エドガー・アラン
HTTP error occurred: 404 Client Error: Not Found for url: https://pubserver2.herokuapp.com/api/v0.1/books/4261/content?format=txt
failed to get data for book id 4

In [173]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
for row in cur2.execute('SELECT difficulty, bookid FROM aozoraStats order by difficulty;'):
    print(row)
con2.close()

(0.53, 50268)
(1.06, 57817)
(1.54, 178)
(1.58, 2674)
(1.66, 59416)
(1.68, 58552)
(1.96, 58102)
(2.05, 52409)
(2.14, 57123)
(2.17, 57816)
(2.22, 57465)
(2.22, 57807)
(2.3, 42620)
(2.33, 47148)
(2.36, 5016)
(2.42, 50328)
(2.53, 58813)
(2.54, 57343)
(2.55, 58853)
(2.58, 45397)
(2.62, 43017)
(2.64, 57147)
(2.69, 2314)
(2.7, 98)
(2.71, 57349)
(2.74, 52395)
(2.78, 57182)
(2.81, 2078)
(2.83, 1098)
(2.87, 42927)
(2.88, 621)
(2.88, 56515)
(2.9, 919)
(2.9, 58455)
(2.93, 59427)
(2.94, 22)
(2.94, 56559)
(2.95, 329)
(2.95, 56078)
(2.96, 682)
(2.99, 2672)
(2.99, 55114)
(2.99, 59429)
(3.02, 48907)
(3.03, 47871)
(3.03, 56400)
(3.04, 140)
(3.05, 894)
(3.06, 2285)
(3.06, 56535)
(3.07, 56641)
(3.09, 17)
(3.11, 530)
(3.18, 2526)
(3.2, 57190)
(3.24, 5018)
(3.24, 42926)
(3.24, 56388)
(3.25, 57187)
(3.26, 1086)
(3.29, 45245)
(3.31, 42309)
(3.31, 59504)
(3.33, 42)
(3.33, 43015)
(3.33, 56648)
(3.34, 42928)
(3.35, 42939)
(3.35, 56406)
(3.38, 56389)
(3.38, 57291)
(3.4, 424)
(3.4, 624)
(3.44, 3814)
(3.44, 49011)


In [174]:
#bookId = 51307 # negative score b/c no periods. uses double newlines.
#bookId = 45630 # negative score b/c no preiods. Uses single newlines.
#bookId = 58278
bookId = 50268
url = f"https://pubserver2.herokuapp.com/api/v0.1/books/{bookId}/content?format=txt"
text = fetchURL(url)
text = CleanUPAozoraText(text)

In [175]:
print(text)

故社員の一言今尚精神
福澤諭吉



　明治元年正月、伏見の変乱、前将軍慶喜公は軍艦に乗て東帰、次で諸方の官軍は問罪として東海東山の諸道より江戸に入り、関東の物論沸くが如く、怒て官兵に抗せんとする者あり、恐れて四方に遁逃する者あり。江戸広しと雖ども、市に売る者なし、家に織る者なし。学者書生の如きもその行く所を知らず、大都会中｜復た一所の学校を見ず、一名の学士に逢わず。独り我慶應義塾の社中は、偶然の発意にして断じて世事に関せず、都下の東南芝新銭座の塾舎に相集りて眠食常に異ならず、弾丸｜雨飛の下、※唔の声を絶たざること殆ど半年、社中自称して戦場中の一小桃源と云いしは蓋しこの時なりき。
　この際に当て府下百万の人民は一時に方向を失い、固より官軍の何ものたるを知らず、仮令い東征の名義｜云々は伝聞するも、その官軍なるものが江戸に入たる上は何等の挙動あるべきや、之を測量すること甚だ易からず。数百年来｜未だ曾て見ざる所の軍事なれば、軍人とあれば必ず乱暴なるものならん、乱暴人は之を避くるに若かずとて、下等社会の群民は無論、上流の士人にても或は俄に家を挙げて藩地に帰る者あり、或は近郷に故旧あれば暫時これに身を寄する者あり。その中に就て独り西洋学者の流は深謀遠慮にして、窃に謂らく、官軍或は暴ならん、仮令い暴なりと雖ども西洋人に害を及ぼすことは彼輩の能する所に非ざるべし、左れば我輩の拠て以て頼む所は横浜にある外国人居留地の安全なるに若くものなしとて、該地に居を移す者日に多く、府民も亦この例に傚うて皆横浜に走り、浜の市中｜既に充満して、その東南なる北方村、本牧村等に及ぼし、一時はその地方にて家賃宿料の騰貴するに至れり。今日在東京の紳士学者にして既往を回想したらば自から之を記臆する輩も多からん、又｜或はその当局者もあらん。
　斯る世上の有様なれば、在江戸の人にして苟も横浜在留の西洋人に知る者あれば、西洋人も亦私に之を保護せんとするの情を抱き、或は仮に某国の籍に入れと云う者あり、或はその印鑑を与えて万一危急のときはこの印鑑を官軍に示して一時を免かれよと云う者あり。何れも皆深切の情に出ることにして、敢て奸策とは云うべからず。我義塾の如きも固より外人に知る者多ければその顧る所と為りて、或る日某氏より態と印鑑を贈り来りしは、全くその友情に出たるものより外ならざるなり。
　時に本塾の教員｜小幡仁三郎小

In [142]:
fetchURL(url)

'〔雨ニモマケズ〕\r\n宮澤賢治\r\n\r\n-------------------------------------------------------\r\n【テキスト中に現れる記号について】\r\n\r\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\r\n\u3000\u3000\u3000（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）\r\n（例）※［＃「「蔭」の「陰のつくり」に代えて「人がしら／髟のへん」、第4水準2-86-78］\r\n-------------------------------------------------------\r\n\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダヲモチ\r\n慾ハナク\r\n決シテ瞋ラズ\r\nイツモシヅカニワラッテヰル\r\n一日ニ玄米四合ト\r\n味噌ト少シノ野菜ヲタベ\r\nアラユルコトヲ\r\nジブンヲカンジョウニ入レズニ\r\nヨクミキキシワカリ\r\nソシテワスレズ\r\n野原ノ松ノ林ノ※［＃「「蔭」の「陰のつくり」に代えて「人がしら／髟のへん」、第4水準2-86-78］ノ\r\n小サナ萓ブキノ小屋ニヰテ\r\n東ニ病気ノコドモアレバ\r\n行ッテ看病シテヤリ\r\n西ニツカレタ母アレバ\r\n行ッテソノ稲ノ朿ヲ［＃「朿ヲ」はママ］負ヒ\r\n南ニ死ニサウナ人アレバ\r\n行ッテコハガラナクテモイヽトイヒ\r\n北ニケンクヮヤソショウガアレバ\r\nツマラナイカラヤメロトイヒ\r\nヒドリノトキハナミダヲナガシ\r\nサムサノナツハオロオロアルキ\r\nミンナニデクノボートヨバレ\r\nホメラレモセズ\r\nクニモサレズ\r\nサウイフモノニ\r\nワタシハナリタイ\r\n\r\n南無無辺行菩薩\r\n南無上行菩薩\r\n南無多宝如来\r\n南無妙法蓮華経\r\n南無釈迦牟尼仏\r\n南無浄行菩薩\r\n南無安立行菩薩\r\n\r\n\r\n\r\n底本：「【新】校本宮澤賢治全集\u3000第十三巻（上）覚書・手帳\u3000本文篇」筑摩書房\r\n\u3000\u3000\u30001997（平成9）年7月30日初版第1刷発行\r\n※本文については写真版を含む本書によった。また、

In [169]:
try:
    # connect to dbs
    con = sqlite3.connect("dbOutput/jmdict.db")
    cur = con.cursor()
    con2 = sqlite3.connect('dbOutput/aozoraStats.db')
    cur2 = con2.cursor()
    tokens = tokenizer_obj.tokenize(text, mode)
    (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) \
        = ExtractStats(tokens, text)

    difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
    print("text len {}, difficulty = {}".format(bookLen, round(difficulty, 2)))

except Exception as err:
    print(f'Error occurred: {err}')
    print(traceback.format_exc())
finally:
    #finally close the connection to jmdictfurigana
    con.close()
    con2.close()

numSingle 46, numDouble 5, use double False
using num of newlines since no periods found
unique words: kango 25, wago 26, verbs 0
result avgLen 2.59, kango 34, wago 39, gairaigo 35 verbs 0, auxiliary 13, auxVerbs 0
text len 19102, difficulty = 6.12


In [158]:
numSingle = text.count(os.linesep)
numDouble = text.count(os.linesep + os.linesep)

In [167]:
single

840

In [160]:
souble

5

In [176]:
longBooks

[773,
 301,
 456,
 789,
 2093,
 57624,
 752,
 1465,
 1565,
 57228,
 794,
 49866,
 43737,
 776,
 56650,
 59300,
 2253,
 52410,
 275,
 47061,
 58093,
 935,
 42286,
 52504,
 51215,
 56866,
 60202,
 69,
 56669,
 56645,
 56646,
 42618,
 4803,
 50909,
 56651,
 816,
 60,
 522,
 52411,
 56143,
 46669,
 1504,
 52016,
 56698,
 52412,
 761,
 52413,
 1502,
 51732,
 1869,
 52414,
 605,
 56647,
 52415,
 52417,
 52419,
 882,
 52416,
 52418,
 42934,
 52420,
 57405,
 4673,
 42621,
 1058,
 57649,
 50131,
 695,
 785,
 1317,
 57612,
 52421,
 57181,
 307,
 42773,
 57105,
 56041,
 647,
 946,
 57788,
 56642,
 1746,
 2282,
 1669,
 57347,
 760,
 4328,
 5017,
 772,
 56461,
 521,
 1737,
 56649,
 201,
 775,
 59519,
 782,
 52396,
 783,
 46322,
 689,
 1566,
 42377,
 56873,
 42600,
 56568,
 255,
 1924,
 765,
 42941,
 57841,
 496,
 859,
 219,
 56146,
 204,
 51731,
 673,
 49862,
 57583,
 59900,
 43522,
 58463,
 59071,
 1154,
 48222,
 5019,
 49296,
 56673,
 57240,
 57567,
 57515,
 1505,
 774,
 5020,
 4618,
 8,
 47057,


In [172]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
sql = 'DELETE FROM aozoraStats where difficulty < 0.5'
cur2 = con2.cursor()
cur2.execute(sql)
con2.commit()
con2.close()